# Overview
- histogram in DL-DT
- count for different DL

In [1]:
import numpy as np
import pandas as pd
import xarray as xr
import pyproj
from rasterio.transform import Affine

import matplotlib.pyplot as plt
import matplotlib.colors as cl
from matplotlib.ticker import FormatStrFormatter

import cartopy.crs as ccrs
import cartopy.feature as cfeature
import cartopy.geodesic as cgeo
crs = ccrs.PlateCarree()
import cmocean.cm as cm

from xgcm import Grid
from xhistogram.xarray import histogram
import warnings
warnings.filterwarnings("ignore")

import os
from glob import glob

import histlib.box as box
import histlib.aviso as aviso
import histlib.cstes as cstes
import histlib.diagnosis as diag
import histlib.erastar as eras
import histlib.stress_to_windterm as stw

from histlib.cstes import labels, zarr_dir, matchup_dir
plt.rcParams.update({'font.size':15})

/home1/datahome/mdemol/.miniconda3/envs/histenv2/lib/python3.9/site-packages/dask/config.py:742: FutureWarning: Dask configuration key 'distributed.scheduler.transition-log-length' has been deprecated; please use 'distributed.admin.low-level-log-length' instead
  warnings.warn(
/home1/datahome/mdemol/.miniconda3/envs/histenv2/lib/python3.9/site-packages/dask/config.py:742: FutureWarning: Dask configuration key 'distributed.comm.recent-messages-log-length' has been deprecated; please use 'distributed.admin.low-level-log-length' instead
  warnings.warn(


In [2]:
import pandas as pd
import numpy as np
from histlib.cstes import labels, zarr_dir

In [3]:
def get_drifter_sat_year_from_label(l):
    L = l.split('_')
    if 'Sentinel' in l:
        return [L[0], L[1]+'_'+L[2], L[-1]]
    else :
        return L

In [4]:
if True:
    from dask.distributed import Client
    from dask_jobqueue import PBSCluster
    cluster = PBSCluster(cores=7, processes=7, walltime='04:00:00')
    w = cluster.scale(jobs=3)
else:
    from dask.distributed import Client, LocalCluster
    #
    cluster = LocalCluster()

client = Client(cluster)
client

Connection method: Cluster object,Cluster type: dask_jobqueue.PBSCluster
Dashboard: http://10.148.1.42:8787/status,
Dashboard: http://10.148.1.42:8787/status,Workers: 0
Total threads: 0,Total memory: 0 B
Comm: tcp://10.148.1.42:33309,Workers: 0
Dashboard: http://10.148.1.42:8787/status,Total threads: 0
Started: Just now,Total memory: 0 B


In [17]:
cluster.close()

NameError: name 'cluster' is not defined

- Geographical repartition
- Number per combinations unfiltered /filtered

In [12]:
zarr_dir_ok = zarr_dir+'_ok'

In [13]:
zarr_dir_ok

'/home/datawork-lops-osi/aponte/margot/historical_coloc_ok'

_________
# Whole dataset 200km-30min

In [6]:
df = pd.DataFrame(data = [[0]*5]*len(labels), columns=['label', 'drifter_type', 'satellite', 'year', 'number_coloc',])
dfd = pd.DataFrame(data = [[0]*5]*len(labels), columns=['label', 'drifter_type', 'satellite', 'year', 'number_coloc',])
i=0
for l in labels:
    dsm = xr.open_dataset(os.path.join(matchup_dir, f'matchup_{l}.zarr')).dropna('obs').chunk({'obs':500})
    dsm= dsm.where(dsm.alti___time_difference<=30*60, drop=True)
    dsmd = dsm.where(dsm.drogue_status, drop=True)
    df.iloc[i,:]=[l]+get_drifter_sat_year_from_label(l)+ [dsm.dims['obs']]
    dfd.iloc[i,:]=[l]+get_drifter_sat_year_from_label(l)+ [dsmd.dims['obs']]
    i+=1
    print(l)


gps_Jason-3_2020
argos_Jason-3_2020
gps_SARAL_2020
argos_SARAL_2020
gps_Cryosat-2_2020
argos_Cryosat-2_2020
gps_Sentinel-3_A_2020
argos_Sentinel-3_A_2020
gps_Sentinel-3_B_2020
argos_Sentinel-3_B_2020
gps_Jason-3_2019
argos_Jason-3_2019
gps_SARAL_2019
argos_SARAL_2019
gps_Cryosat-2_2019
argos_Cryosat-2_2019
gps_Sentinel-3_A_2019
argos_Sentinel-3_A_2019
gps_Sentinel-3_B_2019
argos_Sentinel-3_B_2019
gps_Jason-3_2018
argos_Jason-3_2018
gps_SARAL_2018
argos_SARAL_2018
gps_Cryosat-2_2018
argos_Cryosat-2_2018
gps_Sentinel-3_A_2018
argos_Sentinel-3_A_2018
gps_Sentinel-3_B_2018
argos_Sentinel-3_B_2018
gps_Jason-3_2017
argos_Jason-3_2017
gps_Jason-2_2017
argos_Jason-2_2017
gps_SARAL_2017
argos_SARAL_2017
gps_Cryosat-2_2017
argos_Cryosat-2_2017
gps_Sentinel-3_A_2017
argos_Sentinel-3_A_2017
gps_Jason-3_2016
argos_Jason-3_2016
gps_Jason-2_2016
argos_Jason-2_2016
gps_SARAL_2016
argos_SARAL_2016
gps_Cryosat-2_2016
argos_Cryosat-2_2016
gps_Sentinel-3_A_2016
argos_Sentinel-3_A_2016
gps_Jason-2_2015
arg

In [18]:
df.to_csv(os.path.join(zarr_dir_ok, 'count', 'count_200km_30min.csv'))
dfd.to_csv(os.path.join(zarr_dir_ok, 'count', 'count_200km_30min_drogued.csv'))

In [20]:
df.groupby('drifter_type').sum()

,number_coloc
drifter_type,
argos,2003233
gps,2382924


In [21]:
dfd.groupby('drifter_type').sum()

,number_coloc
drifter_type,
argos,724895
gps,1135100


_________
# Whole dataset 100km-30min

In [15]:
df_ = pd.DataFrame(data = [[0]*5]*len(labels), columns=['label', 'drifter_type', 'satellite', 'year', 'number_coloc',])
dfd_ = pd.DataFrame(data = [[0]*5]*len(labels), columns=['label', 'drifter_type', 'satellite', 'year', 'number_coloc',])
i=0
for l in labels:
    dsm = xr.open_dataset(os.path.join(matchup_dir, f'matchup_{l}.zarr')).dropna('obs').chunk({'obs':500})
    dsm= dsm.where(dsm.alti___distance<=25e3, drop=True)
    dsm= dsm.where(dsm.alti___time_difference<=30*60, drop=True)
    dsmd = dsm.where(dsm.drogue_status, drop=True)
    df_.iloc[i,:]=[l]+get_drifter_sat_year_from_label(l)+ [dsm.dims['obs']]
    dfd_.iloc[i,:]=[l]+get_drifter_sat_year_from_label(l)+ [dsmd.dims['obs']]
    i+=1
    print(l)


gps_Jason-3_2020
argos_Jason-3_2020
gps_SARAL_2020
argos_SARAL_2020
gps_Cryosat-2_2020
argos_Cryosat-2_2020
gps_Sentinel-3_A_2020
argos_Sentinel-3_A_2020
gps_Sentinel-3_B_2020
argos_Sentinel-3_B_2020
gps_Jason-3_2019
argos_Jason-3_2019
gps_SARAL_2019
argos_SARAL_2019
gps_Cryosat-2_2019
argos_Cryosat-2_2019
gps_Sentinel-3_A_2019
argos_Sentinel-3_A_2019
gps_Sentinel-3_B_2019
argos_Sentinel-3_B_2019
gps_Jason-3_2018
argos_Jason-3_2018
gps_SARAL_2018
argos_SARAL_2018
gps_Cryosat-2_2018
argos_Cryosat-2_2018
gps_Sentinel-3_A_2018
argos_Sentinel-3_A_2018
gps_Sentinel-3_B_2018
argos_Sentinel-3_B_2018
gps_Jason-3_2017
argos_Jason-3_2017
gps_Jason-2_2017
argos_Jason-2_2017
gps_SARAL_2017
argos_SARAL_2017
gps_Cryosat-2_2017
argos_Cryosat-2_2017
gps_Sentinel-3_A_2017
argos_Sentinel-3_A_2017
gps_Jason-3_2016
argos_Jason-3_2016
gps_Jason-2_2016
argos_Jason-2_2016
gps_SARAL_2016
argos_SARAL_2016
gps_Cryosat-2_2016
argos_Cryosat-2_2016
gps_Sentinel-3_A_2016
argos_Sentinel-3_A_2016
gps_Jason-2_2015
arg

In [18]:
df_.groupby('drifter_type').sum()

,number_coloc
drifter_type,
argos,251300
gps,301725


In [16]:
df_.to_csv(os.path.join(zarr_dir_ok, 'count', 'count_25km_30min.csv'))
dfd_.to_csv(os.path.join(zarr_dir_ok, 'count', 'count_25km_30min_drogued.csv'))

In [ ]:
cluster.close()